## Setup

In [ ]:
import random
from collections import Counter
import itertools
import math

import networkx as nx
import numpy as np
import pandas as pd
from scipy.special import softmax
from tqdm import trange, tqdm
import matplotlib as mpl

In [ ]:
class Car:
    def __init__(self, source, target) -> None:
        self.source = source
        self.target = target

    def __repr__(self) -> str:
        return f'<Car {self.source} -> {self.target}>'

    def act(self, network, verbose=False):
        if self.source == self.target or not nx.has_path(network, self.source, self.target):
            return self.source
        else:
            choices = list(nx.all_shortest_paths(network, self.source, self.target, weight='anticipated_latency'))
            choice = random.choice(choices)
            
            if verbose:
                print(f'Choose {choice} from {choices}.')
            
            return choice
            

class TrafficModel:
    def __init__(self, network, cars) -> None:
        self.network = network
        self.cars = cars

    def update_latencies(self):
        nx.set_edge_attributes(self.network, {(v, w): attr['latency_fn'](attr['utilization']) for v, w, attr in self.network.edges(data=True)}, 'latency')
        nx.set_edge_attributes(self.network, {(v, w): attr['latency_fn'](attr['utilization'] + (1 / len(self.cars))) for v, w, attr in self.network.edges(data=True)}, 'anticipated_latency')

    def update_latency(self, edge):
        self.network.edges[edge]['latency'] = self.network.edges[edge]['latency_fn'](self.network.edges[edge]['utilization'])
        self.network.edges[edge]['anticipated_latency'] = self.network.edges[edge]['latency_fn'](self.network.edges[edge]['utilization'] + (1 / len(self.cars)))

    @property
    def allowed_network(self):
        return nx.restricted_view(self.network, [], [(v, w) for v, w, allowed in self.network.edges(data='allowed') if not allowed])
    
    def decrease_utilization(self, edge):
        self.set_utilization(edge, self.network.edges[edge]['utilization'] - (1 / len(self.cars)))

    def increase_utilization(self, edge):
        self.set_utilization(edge, self.network.edges[edge]['utilization'] + (1 / len(self.cars)))

    def set_utilization(self, edge, utilization):
        self.network.edges[edge]['utilization'] = utilization
        self.update_latency(edge)

    def set_edge_restriction(self, edge, allowed=True):
        self.network.edges[edge]['allowed'] = allowed

In [ ]:
def run(model, number_of_steps, verbose=False):
    routes = {id: [] for id in model.cars}

    nx.set_edge_attributes(model.network, 0, 'utilization')
    model.update_latencies()

    stats = []
    stats2 = []
    for step in (range if verbose else trange)(number_of_steps):
        if verbose:
            print(f'Step {step}:')
            print(f'Initial latency = {nx.get_edge_attributes(model.network, "latency")}')

        for id, car in np.random.permutation(list(model.cars.items())):
            for edge in zip(routes[id], routes[id][1:]):
                model.decrease_utilization(edge)

            # Let agents choose a route, given the network with allowed edges only
            routes[id] = car.act(model.allowed_network, verbose=verbose)

            for edge in zip(routes[id], routes[id][1:]):
                model.increase_utilization(edge)

        stats.append({
            **routes, 
            **nx.get_edge_attributes(model.network, 'latency'), 
            **{f't({id})': nx.path_weight(model.network, route, 'latency') for id, route in routes.items()}})

        for id, car in np.random.permutation(list(model.cars.items())):
            stats2.append({'step': step, 'car_id': id, 'source': car.source, 'target': car.target, 'route': tuple(routes[id]), 'travel_time': nx.path_weight(model.network, routes[id], 'latency')})

    return pd.DataFrame(stats), pd.DataFrame(stats2)

In [ ]:
def build_network(network):
    # Create latency functions from parameters
    nx.set_edge_attributes(network, {(v, w): lambda u, a=attr['latency_params'][0], b=attr['latency_params'][1], c=attr['latency_params'][2]: a + b * u ** c for v, w, attr in network.edges(data=True)}, 'latency_fn')

    # Set initial utilization
    nx.set_edge_attributes(network, 0, 'utilization')

    # Set initial latency based on utilization
    nx.set_edge_attributes(network, {(v, w): attr['latency_fn'](attr['utilization']) for v, w, attr in network.edges(data=True)}, 'latency')

    # Allow all edges
    nx.set_edge_attributes(network, True, 'allowed')

    return network

In [ ]:
def plot_travel_time_per_route(car_stats):
    car_stats.groupby('route')['travel_time'].plot(legend=True)

In [ ]:
def plot_cars_per_edge(car_stats):
    pd.DataFrame(list(car_stats.groupby('step')['route'].aggregate(lambda routes: Counter([edge for route in routes for edge in tuple(zip(route, route[1:]))])))).plot()

In [ ]:
def plot_cars_per_route(car_stats):
    car_stats.groupby(['step', 'route']).size().unstack().plot()

## Braess Paradox

In [ ]:
def create_braess_network():
    network = nx.DiGraph([(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)])

    nx.set_node_attributes(network, {i: (i / 3, 0) for i in range(len(network.nodes))}, 'position')

    # Latency is defined in terms of the utilization, i.e., the share of cars on a specific road
    nx.set_edge_attributes(network, {
        (0, 1): (0, 8, 1),
        (0, 2): (11, 0, 1),
        (1, 2): (1, 0, 1),
        (1, 3): (11, 0, 1),
        (2, 3): (0, 8, 1)
    }, 'latency_params')

    return build_network(network)

### Basic version

In [ ]:
number_of_cars = 20
number_of_steps = 20

network = create_braess_network()
cars = {id: Car(0, 3) for id in range(number_of_cars)}

model = TrafficModel(network, cars)

model.set_edge_restriction((1, 2), allowed=True)
stats, car_stats = run(model, number_of_steps, verbose=False)

# Calculate social welfare as mean agent reward per step
stats['social_welfare'] = stats[[f't({id})' for id in range(number_of_cars)]].mean(axis=1)

In [ ]:
stats['social_welfare'].plot()

In [ ]:
# Number of cars per route over all time steps
plot_cars_per_route(car_stats)

In [ ]:
# Number of cars per edge over all time steps
plot_cars_per_edge(car_stats)

In [ ]:
def plot_latency_increase_per_edge(car_stats):
    # edge_latency = (pd.DataFrame(list(stats2.groupby('step')['route'].aggregate(lambda routes: Counter([edge for route in routes for edge in tuple(zip(route, route[1:]))])))) / len(model.cars)).mean().to_dict()
    # edge_utilization_std = (pd.DataFrame(list(stats2.groupby('step')['route'].aggregate(lambda routes: Counter([edge for route in routes for edge in tuple(zip(route, route[1:]))])))) / len(model.cars)).std().to_dict()

    # if show_std:
    #     edge_labels = {edge: f'{edge_utilization[edge]:.2f} (+- {edge_utilization_std[edge]:.1f})' for edge in edge_utilization}
    # else:
    #     edge_labels = {edge: f'{edge_utilization[edge]:.2f}' for edge in edge_utilization}

    # nx.draw(model.network, pos=nx.get_node_attributes(model.network, 'position'), edgelist=edge_utilization.keys(), edge_color=edge_utilization.values(), edge_vmax=1.0, edge_cmap=mpl.colormaps['Greys'])
    # nx.draw_networkx_edge_labels(model.network, pos=nx.get_node_attributes(model.network, 'position'), edge_labels=edge_labels)

### With random initialization of cars

In [ ]:
number_of_cars = 20
number_of_steps = 20

network = create_braess_network()
cars = {id: Car(s, t) for id, (s, t) in zip(range(number_of_cars), random.choices([(0, 2), (0, 3), (1, 3)], k=number_of_cars))}
print(f'Cars: {cars}')

model = TrafficModel(network, cars)

model.set_edge_restriction((1, 2), allowed=True)
stats, car_stats = run(model, number_of_steps, verbose=False)

# Calculate social welfare as mean agent reward per step
stats['social_welfare'] = stats[[f't({id})' for id in range(number_of_cars)]].mean(axis=1)

In [ ]:
# Social welfare (i.e., mean travel time) over all time steps
stats['social_welfare'].plot()

In [ ]:
# Travel time per route over all time steps
plot_travel_time_per_route(car_stats)

In [ ]:
# Number of cars per route over all time steps
plot_cars_per_route(car_stats)

In [ ]:
# Number of cars per edge over all time steps
plot_cars_per_edge(car_stats)

## Grid with random latency functions

In [ ]:
def create_random_grid_network(number_of_rows, number_of_columns):
    network = nx.grid_2d_graph(number_of_rows, number_of_columns, create_using=nx.DiGraph)

    nx.set_node_attributes(network, {(i, j): (j / (number_of_columns - 1), 1 - (i / (number_of_rows - 1))) for i, j in network.nodes}, 'position')
    nx.set_edge_attributes(network, {edge: random.choice([(1, 0, 1), (11, 0, 1), (0, 8, 1)]) for edge in network.edges}, 'latency_params')

    return build_network(network)

In [ ]:
def draw_edge_utilization(model, stats2, show_std=False):
    edge_utilization = (pd.DataFrame(list(stats2.groupby('step')['route'].aggregate(lambda routes: Counter([edge for route in routes for edge in tuple(zip(route, route[1:]))])))) / len(model.cars)).mean().to_dict()
    edge_utilization_std = (pd.DataFrame(list(stats2.groupby('step')['route'].aggregate(lambda routes: Counter([edge for route in routes for edge in tuple(zip(route, route[1:]))])))) / len(model.cars)).std().to_dict()

    if show_std:
        edge_labels = {edge: f'{edge_utilization[edge]:.2f} (+- {edge_utilization_std[edge]:.1f})' for edge in edge_utilization}
    else:
        edge_labels = {edge: f'{edge_utilization[edge]:.2f}' for edge in edge_utilization}

    nx.draw(model.network, pos=nx.get_node_attributes(model.network, 'position'), edgelist=edge_utilization.keys(), edge_color=edge_utilization.values(), edge_vmax=1.0, edge_cmap=mpl.colormaps['Greys'])
    nx.draw_networkx_edge_labels(model.network, pos=nx.get_node_attributes(model.network, 'position'), edge_labels=edge_labels)

In [ ]:
def draw_latency_params(model):
    nx.draw(model.network, pos=nx.get_node_attributes(model.network, 'position'))
    nx.draw_networkx_edge_labels(model.network, pos=nx.get_node_attributes(model.network, 'position'), edge_labels={(v, w): f'{model.network.edges[(v, w)]["latency_params"]}\n{model.network.edges[(w, v)]["latency_params"]}' for v, w in model.network.edges if v < w});

In [ ]:
number_of_rows = 4
number_of_columns = 5
number_of_cars = 100
number_of_steps = 100

network = create_random_grid_network(number_of_rows, number_of_columns)
cars = {id: Car((0, 0), (number_of_rows - 1, number_of_columns - 1)) for id in range(number_of_cars)}

model = TrafficModel(network, cars)
stats, car_stats = run(model, number_of_steps)

# Calculate social welfare as mean agent reward per step
stats['social_welfare'] = stats[[f't({id})' for id in range(number_of_cars)]].mean(axis=1)

In [ ]:
draw_latency_params(model)

In [ ]:
stats['social_welfare'].plot()

In [ ]:
stats2.groupby(['step', 'route']).size().unstack().plot()

In [ ]:
# Travel time per route over all time steps

stats2.groupby('route')['travel_time'].plot(legend=True);

In [ ]:
# Number of cars per edge over all time steps

pd.DataFrame(list(stats2.groupby('step')['route'].aggregate(lambda routes: Counter([edge for route in routes for edge in tuple(zip(route, route[1:]))])))).rolling(5).mean().plot()

In [ ]:
draw_edge_utilization(model, stats2, show_std=False)